In [27]:
from connections import bb_session
bb_s = bb_session()

# Get Course Memberships
The LTI Tool does the following
* Pull membership information
* Create a group for each unique child course
* Enroll members in groups 

In [46]:
# Pull membership information from parent course
parentCourse = 'TST-Merge01'
memberships = bb_s.get(f'v1/courses/courseId:{parentCourse}/users').json()['results']
memberships = [(membership['userId'], membership.get('childCourseId', membership.get('courseId'))) 
                 for membership in memberships 
                 if membership['courseRoleId'] == 'Student']

In [47]:
# Create a group for each unique child course
coursePks = set([member[1] for member in memberships])
for courseId in coursePks:
    course_info = bb_s.get(f'v2/courses/{courseId}').json()
    group = {
        'name':course_info['courseId'],
        'externalId':course_info['courseId'],
        'availability':{'available':'Yes'}
    }
    r = bb_s.post(f'v1/courses/courseId:{parentCourse}/groups', json=group)
    print(r.status_code)

409
409


In [48]:
# Enroll Members in Groups
for member in memberships:
    userId = member[0]
    courseId = member[1]
    groupId = bb_s.get(f'v2/courses/{courseId}').json()['courseId']
    r = bb_s.put(f'v2/courses/courseId:{parentCourse}/groups/externalId:{groupId}/users/{userId}')
    print(r.status_code)

500
500
500
500
